In [ ]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

In [ ]:
from modules.data_processing import DatasetProcessor
from modules.bert_data_processing import BERTDataProcessor
from modules.bert_evaluator import BERTEvaluator
from modules.bert_trainer import BERTTrainer
import json
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG

In [3]:
def train_model(model_name, dataset_name, epochs=3, batch_size=16):
    if dataset_name.lower() == 'combined':
      ewt_processor = DatasetProcessor('ewt', from_json=True)
      edt_processor = DatasetProcessor('edt', from_json=True)
      ewt_dataset = DatasetProcessor.tag_to_id(ewt_processor.dataset, TAG2IDX)
      edt_dataset = DatasetProcessor.tag_to_id(edt_processor.dataset, TAG2IDX)
      dataset = DatasetProcessor.combine_datasetdicts(ewt_dataset, edt_dataset)
    elif dataset_name.lower() in ['ewt', 'edt']:
      processor = DatasetProcessor(dataset_name.lower(), from_json=True)
      dataset = DatasetProcessor.tag_to_id(processor.dataset, TAG2IDX)

    print(f'{dataset_name.upper()} andmestik laetud')
    bert_processor = BERTDataProcessor(model_name)
    evaluator = BERTEvaluator(all_tags=ALL_TAGS)

    tokenized_dataset = bert_processor.tokenize_dataset(dataset)

    trainer = BERTTrainer(model_name=model_name, idx2tag=IDX2TAG, tag2idx=TAG2IDX, evaluator=evaluator)

    model, model_trainer = trainer.finetune_model(processor=bert_processor, tokenized_dataset=tokenized_dataset, epochs=epochs, batch_size=batch_size, early_stop_patience=3, output_dir=f'./results/models/{model_name.split("/")[1]}/{dataset_name}/trained_for_{epochs}')

    results = evaluator.evaluate_and_print(tokenized_dataset['test'], model_trainer)
    evaluator.evaluation_to_json(nervaluate_strict_overall=results[0]['strict'], nervaluate_by_tag=results[1], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on=dataset_name, epochs=epochs)

    if dataset_name.lower() == 'combined':
      tokenized_ewt = bert_processor.tokenize_dataset(ewt_dataset['test'])
      tokenized_edt = bert_processor.tokenize_dataset(edt_dataset['test'])

      print(f'Kombineeritud andmestikul treenitud {model_name} EWT testandmestikul')
      ewt_results = evaluator.evaluate_and_print(tokenized_ewt, model_trainer)
      evaluator.evaluation_to_json(nervaluate_strict_overall=ewt_results[0]['strict'], nervaluate_by_tag=ewt_results[1], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on="EWT", epochs=epochs)

      print(f'Kombineeritud andmestikul treenitud {model_name} EDT testandmestikul')
      edt_results = evaluator.evaluate_and_print(tokenized_edt, model_trainer)
      evaluator.evaluation_to_json(nervaluate_strict_overall=edt_results[0]['strict'], nervaluate_by_tag=edt_results[1], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on="EDT", epochs=epochs)

      return model, model_trainer, (results, ewt_results, edt_results)

    return model, model_trainer, results

def train_all():
  dataset_names = ['EWT', 'EDT', 'Combined']

  model_names = ["EMBEDDIA/est-roberta", "tartuNLP/EstBERT"] # "tartuNLP/EstRoBERTa" XLM-RoBERTa eestikeelne variant, aga puudub igasugune taustainfo
  results_dict = {}

  for model_name in model_names:
    model_results = {}
    for dataset_name in dataset_names:
      model, model_trainer, results = train_model(model_name, dataset_name, epochs=3)
      model_results[dataset_name] = results
    results_dict[model_name] = model_results

  return results_dict

In [ ]:
def main():
    results = train_all()

In [ ]:
main()

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

EWT andmestik laetud


tokenizer_config.json:   0%|          | 0.00/509 [00:00<?, ?B/s]

...

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


Nervaluate tulemused
Strict {'correct': 1263, 'incorrect': 266, 'partial': 0, 'missed': 326, 'spurious': 165, 'possible': 1855, 'actual': 1694, 'precision': 0.7455726092089728, 'recall': 0.6808625336927224, 'f1': 0.7117497886728656}
precision 0.7455726092089728
recall 0.6808625336927224
f1 0.7117497886728656
EVE {'correct': 33, 'incorrect': 5, 'partial': 0, 'missed': 19, 'spurious': 13, 'possible': 57, 'actual': 51, 'precision': 0.6470588235294118, 'recall': 0.5789473684210527, 'f1': 0.6111111111111113}
GEP {'correct': 41, 'incorrect': 10, 'partial': 0, 'missed': 4, 'spurious': 2, 'possible': 55, 'actual': 53, 'precision': 0.7735849056603774, 'recall': 0.7454545454545455, 'f1': 0.7592592592592593}
LOC {'correct': 34, 'incorrect': 12, 'partial': 0, 'missed': 9, 'spurious': 15, 'possible': 55, 'actual': 61, 'precision': 0.5573770491803278, 'recall': 0.6181818181818182, 'f1': 0.5862068965517241}
MUU {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'a

Nervaluate tulemused
Strict {'correct': 3640, 'incorrect': 711, 'partial': 0, 'missed': 212, 'spurious': 294, 'possible': 4563, 'actual': 4645, 'precision': 0.7836383207750269, 'recall': 0.7977207977207977, 'f1': 0.790616854908775}
precision 0.7836383207750269
recall 0.7977207977207977
f1 0.790616854908775
EVE {'correct': 33, 'incorrect': 22, 'partial': 0, 'missed': 29, 'spurious': 10, 'possible': 84, 'actual': 65, 'precision': 0.5076923076923077, 'recall': 0.39285714285714285, 'f1': 0.4429530201342282}
GEP {'correct': 297, 'incorrect': 87, 'partial': 0, 'missed': 0, 'spurious': 15, 'possible': 384, 'actual': 399, 'precision': 0.7443609022556391, 'recall': 0.7734375, 'f1': 0.7586206896551724}
LOC {'correct': 317, 'incorrect': 111, 'partial': 0, 'missed': 13, 'spurious': 8, 'possible': 441, 'actual': 436, 'precision': 0.7270642201834863, 'recall': 0.7188208616780045, 'f1': 0.7229190421892816}
MUU {'correct': 0, 'incorrect': 5, 'partial': 0, 'missed': 1, 'spurious': 0, 'possible': 6, 'ac

In [6]:
estroberta_edt, estroberta_edt_trainer, estroberta_edt_results = train_model('EMBEDDIA/est-roberta', 'EDT', epochs=6)

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

EDT andmestik laetud


Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

...

[2025-03-24 13:35:07] Alustan EMBEDDIA/est-roberta treenimist


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.081300,0.086085,0.756475,0.758401,0.757437,0.974021,0.580000,0.408451,0.479339,71,0.649438,0.713580,0.680000,405,0.783929,0.672282,0.723825,653,0.000000,0.000000,0.000000,9,0.599613,0.824468,0.694289,376,0.912357,0.917853,0.915097,1826,0.446125,0.437037,0.441534,540,0.000000,0.000000,0.000000,48
2,0.050300,0.094608,0.757827,0.745672,0.751700,0.974515,0.434211,0.464789,0.448980,71,0.554828,0.837037,0.667323,405,0.790909,0.532925,0.636780,653,0.000000,0.000000,0.000000,9,0.595289,0.739362,0.659549,376,0.913620,0.944140,0.928629,1826,0.539062,0.383333,0.448052,540,0.000000,0.000000,0.000000,48
3,0.026400,0.111319,0.755539,0.764002,0.759747,0.974324,0.457831,0.535211,0.493506,71,0.578450,0.755556,0.655246,405,0.753571,0.646248,0.695796,653,0.000000,0.000000,0.000000,9,0.600427,0.747340,0.665877,376,0.919873,0.955641,0.937416,1826,0.480460,0.387037,0.428718,540,0.000000,0.000000,0.000000,48
4,0.015400,0.120865,0.769900,0.765784,0.767837,0.975169,0.558824,0.535211,0.546763,71,0.561258,0.837037,0.671952,405,0.805369,0.551302,0.654545,653,0.000000,0.000000,0.000000,9,0.723861,0.718085,0.720961,376,0.940502,0.943593,0.942045,1826,0.486865,0.514815,0.500450,540,0.000000,0.000000,0.000000,48
5,0.008500,0.119126,0.773638,0.776986,0.775308,0.976125,0.590909,0.549296,0.569343,71,0.604790,0.748148,0.668874,405,0.774566,0.615620,0.686007,653,0.000000,0.000000,0.000000,9,0.609053,0.787234,0.686775,376,0.938841,0.958379,0.948509,1826,0.519841,0.485185,0.501916,540,0.000000,0.000000,0.000000,48
6,0.004300,0.124380,0.779540,0.775967,0.777749,0.976396,0.550000,0.464789,0.503817,71,0.587571,0.770370,0.666667,405,0.784736,0.614089,0.689003,653,0.000000,0.000000,0.000000,9,0.651007,0.773936,0.707169,376,0.941145,0.954545,0.947798,1826,0.533865,0.496296,0.514395,540,0.000000,0.000000,0.000000,48


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

[2025-03-24 13:56:48] EMBEDDIA/est-roberta treenimine lõpetatud
Kokku kulus: 1301.16 sekundit (0.36 tundi)
Hindan testandmestikul..



Nervaluate tulemused
Strict {'correct': 3773, 'incorrect': 671, 'partial': 0, 'missed': 153, 'spurious': 156, 'possible': 4597, 'actual': 4600, 'precision': 0.8202173913043478, 'recall': 0.8207526647813792, 'f1': 0.8204849407415461}
precision 0.8202173913043478
recall 0.8207526647813792
f1 0.8204849407415461
EVE {'correct': 23, 'incorrect': 26, 'partial': 0, 'missed': 30, 'spurious': 7, 'possible': 79, 'actual': 56, 'precision': 0.4107142857142857, 'recall': 0.2911392405063291, 'f1': 0.34074074074074073}
GEP {'correct': 260, 'incorrect': 89, 'partial': 0, 'missed': 8, 'spurious': 6, 'possible': 357, 'actual': 355, 'precision': 0.7323943661971831, 'recall': 0.7282913165266106, 'f1': 0.7303370786516854}
LOC {'correct': 357, 'incorrect': 134, 'partial': 0, 'missed': 14, 'spurious': 9, 'possible': 505, 'actual': 500, 'precision': 0.714, 'recall': 0.7069306930693069, 'f1': 0.7104477611940297}
MUU {'correct': 0, 'incorrect': 6, 'partial': 0, 'missed': 0, 'spurious': 1, 'possible': 6, 'actua

In [8]:
estroberta_combined, estroberta_combined_trainer, estroberta_combined_results = train_model('EMBEDDIA/est-roberta', 'Combined', epochs=6)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud EMBEDDIA/est-roberta EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Nervaluate tulemused
Strict {'correct': 1424, 'incorrect': 254, 'partial': 0, 'missed': 190, 'spurious': 123, 'possible': 1868, 'actual': 1801, 'precision': 0.7906718489727929, 'recall': 0.7623126338329764, 'f1': 0.7762333060779504}
precision 0.7906718489727929
recall 0.7623126338329764
f1 0.7762333060779504
EVE {'correct': 44, 'incorrect': 1, 'partial': 0, 'missed': 9, 'spurious': 2, 'possible': 54, 'actual': 47, 'precision': 0.9361702127659575, 'recall': 0.8148148148148148, 'f1': 0.8712871287128713}
GEP {'correct': 41, 'incorrect': 8, 'partial': 0, 'missed': 8, 'spurious': 0, 'possible': 57, 'actual': 49, 'precision': 0.8367346938775511, 'recall': 0.7192982456140351, 'f1': 0.7735849056603773}
LOC {'correct': 34, 'incorrect': 19, 'partial': 0, 'missed': 7, 'spurious': 6, 'possible': 60, 'actual': 59, 'precision': 0.576271186440678, 'recall': 0.5666666666666667, 'f1': 0.5714285714285714}
MUU {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actua


Nervaluate tulemused
Strict {'correct': 3799, 'incorrect': 635, 'partial': 0, 'missed': 163, 'spurious': 193, 'possible': 4597, 'actual': 4627, 'precision': 0.8210503566025502, 'recall': 0.8264085273004134, 'f1': 0.8237207285342585}
precision 0.8210503566025502
recall 0.8264085273004134
f1 0.8237207285342585
EVE {'correct': 34, 'incorrect': 16, 'partial': 0, 'missed': 29, 'spurious': 10, 'possible': 79, 'actual': 60, 'precision': 0.5666666666666667, 'recall': 0.43037974683544306, 'f1': 0.4892086330935252}
GEP {'correct': 268, 'incorrect': 79, 'partial': 0, 'missed': 10, 'spurious': 9, 'possible': 357, 'actual': 356, 'precision': 0.7528089887640449, 'recall': 0.7507002801120448, 'f1': 0.7517531556802244}
LOC {'correct': 375, 'incorrect': 120, 'partial': 0, 'missed': 10, 'spurious': 14, 'possible': 505, 'actual': 509, 'precision': 0.7367387033398821, 'recall': 0.7425742574257426, 'f1': 0.7396449704142013}
MUU {'correct': 0, 'incorrect': 4, 'partial': 0, 'missed': 2, 'spurious': 0, 'poss

In [10]:
estbert_combined, estbert_combined_trainer, estbert_combined_results = train_model('tartuNLP/EstBERT', 'Combined', epochs=6)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Nervaluate tulemused
Strict {'correct': 1266, 'incorrect': 260, 'partial': 0, 'missed': 329, 'spurious': 150, 'possible': 1855, 'actual': 1676, 'precision': 0.7553699284009546, 'recall': 0.6824797843665769, 'f1': 0.7170773152081563}
precision 0.7553699284009546
recall 0.6824797843665769
f1 0.7170773152081563
EVE {'correct': 33, 'incorrect': 6, 'partial': 0, 'missed': 18, 'spurious': 13, 'possible': 57, 'actual': 52, 'precision': 0.6346153846153846, 'recall': 0.5789473684210527, 'f1': 0.6055045871559633}
GEP {'correct': 40, 'incorrect': 11, 'partial': 0, 'missed': 4, 'spurious': 2, 'possible': 55, 'actual': 53, 'precision': 0.7547169811320755, 'recall': 0.7272727272727273, 'f1': 0.7407407407407407}
LOC {'correct': 27, 'incorrect': 19, 'partial': 0, 'missed': 9, 'spurious': 12, 'possible': 55, 'actual': 58, 'precision': 0.46551724137931033, 'recall': 0.4909090909090909, 'f1': 0.47787610619469023}
MUU {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 2, 'possible': 0,


Nervaluate tulemused
Strict {'correct': 3618, 'incorrect': 717, 'partial': 0, 'missed': 228, 'spurious': 282, 'possible': 4563, 'actual': 4617, 'precision': 0.783625730994152, 'recall': 0.7928994082840237, 'f1': 0.788235294117647}
precision 0.783625730994152
recall 0.7928994082840237
f1 0.788235294117647
EVE {'correct': 29, 'incorrect': 26, 'partial': 0, 'missed': 29, 'spurious': 11, 'possible': 84, 'actual': 66, 'precision': 0.4393939393939394, 'recall': 0.34523809523809523, 'f1': 0.38666666666666666}
GEP {'correct': 256, 'incorrect': 121, 'partial': 0, 'missed': 7, 'spurious': 12, 'possible': 384, 'actual': 389, 'precision': 0.6580976863753213, 'recall': 0.6666666666666666, 'f1': 0.6623544631306597}
LOC {'correct': 323, 'incorrect': 104, 'partial': 0, 'missed': 14, 'spurious': 22, 'possible': 441, 'actual': 449, 'precision': 0.7193763919821826, 'recall': 0.7324263038548753, 'f1': 0.7258426966292135}
MUU {'correct': 0, 'incorrect': 4, 'partial': 0, 'missed': 2, 'spurious': 3, 'possib